In [5]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from src.db.models import DbAuthority, DbWork, DbInstance, DbItem
from pyfuseki import FusekiUpdate, FusekiQuery
from pysolr import Solr
from src.schemas.settings import Settings

In [6]:
settings = Settings()

Clean DB

In [8]:
engine = create_engine(
     f"mariadb+mariadbconnector://{settings.db_user}:{settings.db_pass}@{settings.mariadb}:3306/bk")
session = scoped_session(
    sessionmaker(autocommit=False, autoflush=False, bind=engine))

In [11]:
item = session.query(DbItem)

In [13]:
item.delete()

OperationalError: (mariadb.OperationalError) Lock wait timeout exceeded; try restarting transaction
[SQL: DELETE FROM item]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:


session.query(DbAuthority).delete()
session.query(DbItem).delete()
session.query(DbInstance).delete()
session.query(DbWork).delete()


Clean Jena

In [14]:
fuseki = FusekiUpdate(settings.fuseki, 'bk') 

d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""
response = fuseki.run_sparql(d)
response.convert()


{'statusCode': 200, 'message': 'Update succeeded'}

Clean Solr

In [8]:
solrAuthority = Solr(f'{settings.solr}/solr/authority/', timeout=10)
solrAuthority.delete(q="*:*",  commit=True)

solrCatalog = Solr(f'{settings.solr}/solr/catalog/', timeout=10)
solrCatalog.delete(q="*:*",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">307</int>\n</lst>\n</response>\n'